## Notebook summary

In this notebook we will:

⚡ Establish access to a hosted Redshift database instance

⚡ Create tables fed from data in S3 buckets

⚡ Use SQL within Jupyter notebooks through `JupySQL` to create tables, clean the data, and merge the tables. 

⚡ Visualize the resulting merged table with `seaborn`

⚡ Set up a classification model for the data with `sklearn` (`sklearn.ensemble.RandomForestClassifier`) 

⚡ Evaluate the model's results using `sklearn.metrics `

## What is JupySQL

JupySQL is an open source project whose goal is to enable you to run SQL and plot large datasets in Jupyter via a `%sql`, `%%sql`, and `%sqlplot` magics. 

JupySQL is compatible with all major databases (e.g., PostgreSQL, MySQL, SQL Server), data warehouses (e.g., Snowflake, BigQuery, Redshift), and embedded engines (SQLite, and DuckDB).

## What is Amazon Redshift? 

Amazon Redshift uses SQL to analyze structured and semi-structured data across data warehouses, operational databases, and data lakes, using AWS-designed hardware and machine learning to deliver the best price performance at any scale.
![](redshift.png)

## Goal

Example: https://github.com/aws-samples/getting-started-with-amazon-redshift-data-api/blob/main/use-cases/sagemaker-notebook-redshift-ml/redshift_ml_with_data_api.ipynb

⚡ Enable seamless integration to perform EDA within local Jupyter notebooks

⚡ Remove the need to execute outside scripts within Jupyter notebooks when setting up the tables

⚡ Automatically convert SQL query results to `pandas` or `polars` dataframes for further data analysis

## Pre-requisites

1️⃣ AWS Redshift username and password

2️⃣ AWS Redshift host

3️⃣ S3 bucket(s) with data

4️⃣Jupyter notebooks + the JupySQL extension

## Part I: Setting up access to the data, and setting up a database and tables.

## Dependencies

In [ ]:
%pip install jupysql --upgrade sqlalchemy-redshift redshift-connector "sqlalchemy<2" --quiet

## Modules

In [1]:
from dotenv import load_dotenv
from os import environ
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import os

## Access your credentials securely

In [2]:
# Load environment variables
load_dotenv(".env")

user = os.environ.get("REDSHIFT_USERNAME")
password = os.environ.get("REDSHIFT_PASSWORD")
host = os.environ.get("REDSHIFT_HOST")
iamrole = os.environ.get("IAM_role")

## Form the connection string

In [3]:
url = URL.create(
    drivername="redshift+redshift_connector",
    username=user,
    password=password,
    host=host,
    port=5439,
    database="dev",
)

engine = create_engine(url)

## Load the JupySQL extension

In [4]:
%reload_ext sql

Found pyproject.toml from '/Users/macpro/Documents/GitHub/sql/docs/video-material/redshift'

Settings changed:

Config,value
displaycon,False
feedback,True
autopandas,True
named_parameters,True


## Use the engine to initialize access to our Redshift via the alias `redshift`

In [5]:
%sql engine --alias redshift

## Create table `ridership` from data in S3 bucket

In [6]:
%%sql
DROP TABLE IF EXISTS ridership CASCADE;

CREATE TABLE IF NOT EXISTS ridership
( trip_id               INT, 
 trip_duration_seconds INT, 
 trip_start_time       timestamp,
 trip_stop_time        timestamp,
 from_station_name     VARCHAR(50),
 to_station_name       VARCHAR(50),
 from_station_id       SMALLINT,
 to_station_id         SMALLINT,
 user_type             VARCHAR(20));



COPY ridership FROM 
's3://redshift-ml-bikesharing-data/bike-sharing-data/ridership/'
IAM_ROLE 'arn:aws:iam::<accountid>:role/redshift-jupysql-test-read-s3-role'
FORMAT csv IGNOREHEADER 1 DATEFORMAT 'auto' TIMEFORMAT 'auto' REGION 'us-west-2' gzip;



""


## Verifying `ridership` table

In [7]:
%sqlcmd columns -t ridership 


name,type,nullable,default,autoincrement,comment,info
trip_id,INTEGER,True,None,False,None,{'encode': 'az64'}
trip_duration_seconds,INTEGER,True,None,False,None,{'encode': 'az64'}
trip_start_time,TIMESTAMP,True,None,False,None,{'encode': 'az64'}
trip_stop_time,TIMESTAMP,True,None,False,None,{'encode': 'az64'}
from_station_name,VARCHAR(50),True,None,False,None,{'encode': 'lzo'}
to_station_name,VARCHAR(50),True,None,False,None,{'encode': 'lzo'}
from_station_id,SMALLINT,True,None,False,None,{'encode': 'az64'}
to_station_id,SMALLINT,True,None,False,None,{'encode': 'az64'}
user_type,VARCHAR(20),True,None,False,None,{'encode': 'lzo'}


In [8]:
%sqlcmd profile --table ridership 

,trip_id,trip_duration_seconds,trip_start_time,trip_stop_time,from_station_name,to_station_name,from_station_id,to_station_id,user_type
count,3415315,3415315,3415315,3415315,3415315,3415315,2388423,2388423,3415315
unique,3415315,21278,965559,959085,411,411,359,359,4
mean,2644335.0000,986.0000,nan,nan,nan,nan,7123.0000,7122.0000,nan
min,712382,1,nan,nan,nan,nan,7000,7000,nan
max,4581277,6382030,nan,nan,nan,nan,7391,7391,nan


## Create table `weather` from data in S3 bucket

In [9]:
%%sql
DROP TABLE IF EXISTS weather CASCADE;

CREATE TABLE IF NOT EXISTS weather
( longitude_x         DECIMAL(5,2),
 latitude_y          DECIMAL(5,2), 
 station_name        VARCHAR(20),
 climate_id          BIGINT,
 datetime_utc        TIMESTAMP,
 weather_year        SMALLINT,
 weather_month       SMALLINT,
 weather_day         SMALLINT,
 time_utc            VARCHAR(5),
 temp_c              DECIMAL(5,2),
 temp_flag           VARCHAR(1),
 dew_point_temp_c    DECIMAL(5,2),
 dew_point_temp_flag VARCHAR(1),
 rel_hum             SMALLINT,
 rel_hum_flag        VARCHAR(1),
 precip_amount_mm    DECIMAL(5,2),
 precip_amount_flag  VARCHAR(1),
 wind_dir_10s_deg    VARCHAR(10),
 wind_dir_flag       VARCHAR(1),
 wind_spd_kmh        VARCHAR(10),
 wind_spd_flag       VARCHAR(1),
 visibility_km       VARCHAR(10),
 visibility_flag     VARCHAR(1),
 stn_press_kpa       DECIMAL(5,2),
 stn_press_flag      VARCHAR(1),
 hmdx                SMALLINT,
 hmdx_flag           VARCHAR(1),
 wind_chill          VARCHAR(10),
 wind_chill_flag     VARCHAR(1),
 weather             VARCHAR(10));


COPY weather FROM
's3://redshift-ml-bikesharing-data/bike-sharing-data/weather/'
IAM_ROLE 'arn:aws:iam::<accountid>:role/redshift-jupysql-test-read-s3-role'
FORMAT csv IGNOREHEADER 1 DATEFORMAT 'auto' TIMEFORMAT 'auto' REGION 'us-west-2' gzip;


""


## Verifying `weather` table

In [10]:
%sqlcmd columns -t weather 


name,type,nullable,default,autoincrement,comment,info
longitude_x,"NUMERIC(5, 2)",True,None,False,None,{'encode': 'az64'}
latitude_y,"NUMERIC(5, 2)",True,None,False,None,{'encode': 'az64'}
station_name,VARCHAR(20),True,None,False,None,{'encode': 'lzo'}
climate_id,BIGINT,True,None,False,None,{'encode': 'az64'}
datetime_utc,TIMESTAMP,True,None,False,None,{'encode': 'az64'}
weather_year,SMALLINT,True,None,False,None,{'encode': 'az64'}
weather_month,SMALLINT,True,None,False,None,{'encode': 'az64'}
weather_day,SMALLINT,True,None,False,None,{'encode': 'az64'}
time_utc,VARCHAR(5),True,None,False,None,{'encode': 'lzo'}
temp_c,"NUMERIC(5, 2)",True,None,False,None,{'encode': 'az64'}


In [ ]:
%sqlcmd profile --table weather 


## Create table `holiday` from data in S3 bucket

In [ ]:
%%sql
DROP TABLE IF EXISTS holiday CASCADE;

CREATE TABLE IF NOT EXISTS holiday
( holiday_date  DATE,
 description VARCHAR(100));


COPY holiday FROM
's3://redshift-ml-bikesharing-data/bike-sharing-data/holiday/'
IAM_ROLE 'arn:aws:iam::<accountid>:role/redshift-jupysql-test-read-s3-role'
FORMAT csv IGNOREHEADER 1 DATEFORMAT 'auto' TIMEFORMAT 'auto' REGION 'us-west-2' gzip;


## Verifying `holiday` table


In [ ]:
%sqlcmd columns -t holiday 

In [ ]:
%sqlcmd profile --table holiday 

## Create table `ridership_view` - data clean up

In [ ]:
%%sql
CREATE OR REPLACE VIEW ridership_view AS
SELECT
    trip_time, 
    trip_count,
    TO_CHAR(trip_time,'hh24') ::INT trip_hour,
    TO_CHAR(trip_time, 'dd') :: INT trip_day,
    TO_CHAR(trip_time, 'mm') :: INT trip_month,
    TO_CHAR(trip_time, 'yy') :: INT trip_year,
    TO_CHAR(trip_time, 'q') :: INT trip_quarter,
    TO_CHAR(trip_time, 'w') :: INT trip_month_week,
    TO_CHAR(trip_time, 'd') :: INT trip_week_day
FROM  
    (SELECT  
         CASE
           WHEN TRUNC(r.trip_start_time) < '2017-07-01'::DATE
           THEN CONVERT_TIMEZONE('US/Eastern', DATE_TRUNC('hour',r.trip_start_time))
           ELSE DATE_TRUNC('hour',r.trip_start_time)
         END trip_time, COUNT(1) trip_count
     FROM    
         ridership r
     WHERE    r.trip_duration_seconds BETWEEN 60 AND 60 * 60 * 24
     GROUP BY
         1);

CREATE OR REPLACE VIEW weather_view AS
SELECT  
    CONVERT_TIMEZONE('US/Eastern', 
      DATE_TRUNC('hour',datetime_utc)) daytime, 
        ROUND(AVG(temp_c)) temp_c, 
        ROUND(AVG(precip_amount_mm)) precip_amount_mm
FROM weather
GROUP BY 1;



## Verifying ridership_view table


In [ ]:
%sqlcmd columns -t ridership_view 


In [ ]:
%sqlcmd profile --table ridership_view 

## Merging tables as `trip_data`

In [ ]:
%%sql
DROP TABLE IF EXISTS trip_data CASCADE;
DROP TABLE IF EXISTS trip_data;
CREATE TABLE trip_data AS 
SELECT         
    r.trip_time,
    r.trip_count,
    r.trip_hour,
    r.trip_day,
    r.trip_month,
    r.trip_year,
    r.trip_quarter,
    r.trip_month_week,
    r.trip_week_day,
    w.temp_c,
    w.precip_amount_mm,
    CASE
        WHEN h.holiday_date IS NOT NULL THEN 1
        WHEN TO_CHAR(r.trip_time,'D')::INT IN (1,7) THEN 1
        ELSE 0
    END AS is_holiday,
    ROW_NUMBER() OVER (ORDER BY RANDOM()) AS serial_number 
FROM           
  ridership_view r
JOIN weather_view w
  ON ( r.trip_time = w.daytime )
LEFT OUTER JOIN holiday h
  ON ( TRUNC(r.trip_time) = h.holiday_date );


## Verifying table `trip_data`

In [ ]:
%sqlcmd columns -t trip_data 


In [ ]:
%sqlcmd profile --table trip_data 

## Part II: Exploratory data analysis and initial model

## Wrangle and Visualize data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

## Remember our configuration

In [ ]:
%reload_ext sql

In [ ]:
result = %sql select round(trip_count/100) AS rounded_trips, * from trip_data
result.head()

In [ ]:
print(result.isna().sum())
result.dropna(subset=['precip_amount_mm'], inplace=True)

## Visualizing data 

In [ ]:
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.histplot(data=result, x='trip_count')
plt.title("Distribution of number of trips")
plt.show()

In [ ]:
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.histplot(data=result, x='rounded_trips')
plt.title("Distribution of rounded number of trips")
plt.show()

In [ ]:
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.histplot(data=result, x='is_holiday', kde=False)
plt.title("Distribution of trips (based on whether it was a holiday or not)")
plt.xlabel("Is holiday (0=No, 1=Yes)")
plt.show()

In [ ]:
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.scatterplot(data=result, 
                x= 'temp_c', 
                y='rounded_trips', 
                hue='is_holiday')
plt.xlabel("Temperature (C)")
plt.ylabel("Rounded number of trips")
plt.title("Temperature vs rounded number of trips")
plt.show()

In [ ]:
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.scatterplot(data=result, 
                x= 'precip_amount_mm', 
                y='rounded_trips', 
                hue='is_holiday')
plt.xlabel("Precipitation amount (mm)")
plt.ylabel("Rounded number of trips")

plt.title("Precipitation amount vs rounded number of trips")
plt.show()

## Set up classification model

We're going to create a binary column `low_trip_count` to account for days in which there were less than 200 trips, or more than 200 trips.

Our goal will be to classify whether certain days will have less than 200 trips (`low_trip_count==True`) or if they have 200 trips or more (`low_trip_count==False`).

## Pre-requisites

In [ ]:
!pip install sklearn-evaluation scikit-learn --quiet

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn_evaluation import plot, table

## Set up variables, split data

In [ ]:
# Create column
result['low_trip_count'] = np.where(result['trip_count']<200, True, False)

# Set up X and y variables
X = result[['trip_hour', 'trip_day',
               'trip_month', 'trip_year', 'trip_month_week',
               'trip_week_day', 'temp_c', 'precip_amount_mm', 'is_holiday']]
y = result['low_trip_count']

# Split data - notice class imbalance
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    stratify=y,
                                                    random_state=42)

## Initialize model

In [ ]:
ln = RandomForestClassifier()

ln.fit(X_train, y_train)
y_pred = ln.predict(X_test)

## Evaluate model

In [ ]:
print("Acc on test data: {:,.3f}".format(ln.score(X_test, y_test)))
print(classification_report(y_pred=y_pred, y_true=y_test))

## Evaluate model

In [ ]:
plt.figure(figsize=(20,7))
sns.set(font_scale=1)
plot.confusion_matrix(y_test, y_pred);
plt.show()

In [ ]:
plot.feature_importances(ln, top_n=5, feature_names=list(ln.feature_names_in_))


## Key takeaways

⚡ We used the Jupyter notebook extension `JupySQL` to enable access to a Redshift instance through credentials and a host URL

⚡ With `JupySQL`, we were able to customize the default behaviour of the query results to be of format `pandas` (or `polars`)

⚡ Furthermore, with `JupySQL` we were able to create each of the tables from an S3 bucket, verify that the tables were loaded and get summary stats

⚡ We were then able to use our query results in `pandas` format, and leverage techniques in classic supervised learning to classify whether a given day will have low number of trips (less than 200) or higher. 



## Find us on GitHub



<center>
    <h1>
Learn more about JupySQL 
    </h1>
    <h2>
 🐙 https://github.com/ploomber/jupysql 🐙
    </h2>
    <h3 font-family="courier">
    pip install jupysql
        <h3>
</center>